In [2]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report
import os

# ── 경로/하이퍼파라미터 ──────────────────────────────────────────────
base_dir   = './Test_File'   # 클래스별 하위폴더 포함
img_size   = (224, 224)
batch_size = 32
epochs     = 20
num_classes = 3                         # 클래스 개수

# ── 데이터 제너레이터 (훈련/검증 8:2 분할) ─────────────────────────────
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',       # 학습용
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',     # 검증용
    shuffle=False
)


# 클래스 정보
class_indices = train_generator.class_indices       # {'classA':0, ...}
idx_to_class = [None]*len(class_indices)
for k, v in class_indices.items():
    idx_to_class[v] = k
num_classes = len(idx_to_class)
print(f"\n[INFO] 감지된 클래스 수: {num_classes} → {idx_to_class}")




# ── 사전학습 모델(MobileNetV2) + 커스텀 분류기 ────────────────────────
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 특성 추출 모드

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ── 콜백 (EarlyStopping, 체크포인트) ──────────────────────────────────
early_stop  = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')
model_path  = 'best_animal_model.keras'
checkpoint  = ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss')

# ── 학습 ───────────────────────────────────────────────────────────────
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stop, checkpoint]
)

# ── 평가/예측 및 리포트 ───────────────────────────────────────────────
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes
class_labels = list(val_generator.class_indices.keys())

print(classification_report(y_true, y_pred_classes, target_names=class_labels))


Found 254 images belonging to 4 classes.
Found 62 images belonging to 4 classes.

[INFO] 감지된 클래스 수: 4 → ['Weezing 복사본', 'Wigglytuff 복사본', 'Zapdos 복사본', 'Zubat 복사본']
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 398ms/step - accuracy: 0.7283 - loss: 0.7405 - val_accuracy: 0.9516 - val_loss: 0.1927
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 301ms/step - accuracy: 0.9488 - loss: 0.1639 - val_accuracy: 0.9839 - val_loss: 0.0663
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 324ms/step - accuracy: 0.9764 - loss: 0.0659 - val_accuracy: 0.9839 - val_loss: 0.0504
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - accuracy: 0.9921 - loss: 0.0251 - val_accuracy: 0.9677 - val_loss: 0.0611
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 342ms/step - accuracy: 0.9921 - loss: 0.0228 - val_accuracy: 0.9839 - val_loss: 0.0351
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 340ms/step - accuracy: 1.0000 - loss: 0.0103 - val_accuracy: 0.9839 - val_loss: 0.0339
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 299ms/step - accurac